In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
#train_data[1]

Padding or triming review length to MAXLEN

In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 62s 97ms/step - loss: 0.4196 - acc: 0.8074 - val_loss: 0.2960 - val_acc: 0.8828
Epoch 2/10
625/625 [==============================] - 60s 96ms/step - loss: 0.2406 - acc: 0.9094 - val_loss: 0.2794 - val_acc: 0.8830
Epoch 3/10
625/625 [==============================] - 60s 97ms/step - loss: 0.1862 - acc: 0.9327 - val_loss: 0.3736 - val_acc: 0.8722
Epoch 4/10
625/625 [==============================] - 61s 97ms/step - loss: 0.1527 - acc: 0.9456 - val_loss: 0.2749 - val_acc: 0.8922
Epoch 5/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1306 - acc: 0.9540 - val_loss: 0.3100 - val_acc: 0.8926
Epoch 6/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1139 - acc: 0.9607 - val_loss: 0.3330 - val_acc: 0.8822
Epoch 7/10
625/625 [==============================] - 60s 97ms/step - loss: 0.0970 - acc: 0.9677 - val_loss: 0.4063 - val_acc: 0.8666
Epoch 8/10
625/625 [==============================] - 60s 96ms

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 17s 21ms/step - loss: 0.5174 - acc: 0.8557
[0.5174273252487183, 0.855679988861084]


Check own individual predictions

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

Decode the input numbers to review

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.89931834]
[0.25249827]
